In [ ]:
# For the v3 will try to do a better explanatory data analysis
# Now I will try to use everything given in the dataset by encoding non numerical data

In [1]:
# Importing the libraries for the data analysis
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [18]:
train_data = pd.read_csv('train.csv')

In [19]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [20]:
train_data[train_data['Embarked'].isnull()]



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [21]:
train_data['Embarked'].fillna('S', inplace=True)

/var/folders/zs/58bkf3xx3wx2625zw8xrl_f40000gn/T/ipykernel_60562/3342620763.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Embarked'].fillna('S', inplace=True)


In [23]:
label_encoder = LabelEncoder()
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])

In [24]:
num_train_data = train_data[['Age', 'Pclass', 'SibSp', 'Fare', 'Survived', 'Parch', 'PassengerId', 'Sex']]
# Calculate the correlation matrix
df_all_corr = num_train_data.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()

# Rename columns for better readability
df_all_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)

# Filter the results for correlations involving the 'Age' feature
age_corr = df_all_corr[df_all_corr['Feature 1'] == 'Age']

# Display the filtered correlations
print(age_corr)

   Feature 1    Feature 2  Correlation Coefficient
0        Age          Age                 1.000000
15       Age       Pclass                 0.369226
18       Age        SibSp                 0.308247
27       Age        Parch                 0.189119
37       Age         Fare                 0.096067
39       Age          Sex                 0.093254
44       Age     Survived                 0.077221
51       Age  PassengerId                 0.036847


In [25]:
age_by_pclass_sex = num_train_data.groupby(['Sex', 'Pclass']).median()['Age']

# Display the median ages for each group
for pclass in range(1, 4):
    for sex in [0, 1]:
        print('Median age of Pclass {} {}s: {}'.format(pclass, 'male' if sex == 1 else 'female', age_by_pclass_sex[sex][pclass]))

# Calculate and print the overall median age
print('Median age of all passengers: {}'.format(num_train_data['Age'].median()))

Median age of Pclass 1 females: 35.0
Median age of Pclass 1 males: 40.0
Median age of Pclass 2 females: 28.0
Median age of Pclass 2 males: 30.0
Median age of Pclass 3 females: 21.5
Median age of Pclass 3 males: 25.0
Median age of all passengers: 28.0


In [26]:
train_data['Age'] = train_data.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

In [ ]:
# Age and Embarked are done now we have to deal with the empty Cabin values